<h3>USFWS EDA Pea_Island_Beach_Grain_Size_Data_Global_Analysis </3>

Computes the overall or global mean, standard deviation (sorting), skewness, and kurtosis for all of the beach sand samples collected, over the three-year, twelve survey duration of the Pea Island Beach Monitoring Project. The project spaned from July, 2014 to August, 2017. Sand samples were collected along 27 transects with 4, and later 5, samples per transect. For the entire project, this amounts to more than 1400 individual sand samples.

Each sand sample was processed using standard dry sieving techniques, with mesh spacing, bin sie, set at:

phis=['phi-1','phi-0.5','phi_0','phi_0.5','phi_1','phi_1.25','phi_1.5','phi_1.75','phi_2','phi_2.5','phi_3','phi_3.5','phi_4','remainder']

The resulting individual bin, or sieved fraction weights were recorded in Microsoft Excel spreadsheets--12 in all. In this notebook we extract those sample weights, bring them together and then compute, sample by sample, the four statistical moments, recording each of these in a separate pandas dataframe. The resulting statistics are then pooled and mean values computed for each pooled statistic, yielding the final global values: a pooled sample size mean, standard eviation (sorting), skewness, and kurtosis for the entire 3-year long project.


**SUPPORTS (Not all of this stuff is used/needed):**

transects=['C11','C10','C9','C8','C7','C6','C5','C4','C3','C2','C1','T1','T2','T3','T4','T5','T6','T7','T8','T9','T10',
'T11','T12','T13','T14','T15','T16']

surveys=[201407, 201409, 201504, 201508, 201602, 201605, 201608, 201610 ]


whole phis=['phi_-1','phi_0','phi_1','phi_2','phi_3','remainder']

samples=['S1','S2','S3','S5','S4']
<br /><br />

--Notebook created circa: 1/2018
Author: Paul P


<h4>Load the requisite modules and libraries</h4>

In [1]:
# import requiste Python libraries:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append('/Users/paulp/GoogleDrive/projects/SedSAS/')
import SedSASClass

%matplotlib inline

<h4>User Inputs:</h4>

In [1]:
dflist=[]
dflist_bu=[]

### comment this next line out once the list is created in order to minimize chances of
### accientially overwriting in the middle of loading it with the survey sand sample weights
####### globalL=[]     

<h3>Load data from source into a pandas dataframe</h3>

In [2]:
fp='/Users/paulp/GoogleDrive/projects/PeaIslandBeachMonitoring/data/FinalDataSets/'

surveys=[201407, 201409, 201504, 201508, 201510, 201602, 201605, 201608, 201610, 
         201702, 201704, 201708 ]
transects=['C11','C10','C9','C8','C7','C6','C5','C4','C3','C2','C1','T1','T2','T3','T4',
           'T5','T6','T7','T8','T9','T10','T11','T12','T13','T14','T15','T16']
samples=['S1','S2','S3','S4','S5']  
scrns=[-1.0,-0.5,0.0,0.5,1.0,1.25,1.5,1.75,2.0,2.5,3.0,3.5,4.0,5.0]

###############################################################################

d={}
for survey in surveys:
    fn='PINWR_'+str(survey)+'_GrainSize.xlsx'
    df_=pd.read_excel(fp+fn, header=0)
    
    ### delete all non-essential columns and handle known issues:
    df_=df_.drop(['sheet_code','sample_date','pan_weight','pan+wet_weight',
                'pan+dry_weight'], axis=1).copy()
    df_=df_.rename(columns = {'transect_id':'Transect', 'sample_number':'Sample'})
    df_=df_.replace('-',0).copy()
    df_=df_.replace('.',0).copy()
    
    ### load prepped dataframe to dictionary, d:
    d[survey]=df_

### Compute the Mean, Sorting, Skewness, and Kurtosis for each of the 14+ Samples in dataframe Cheese

Use SedSAS to do the computations, one sample record at a time...

In [4]:
####### For each transect, for each survey, compute the mean, sorting, skewness and
####### kurtosis. The put the output into a Python dictionary--we'll build a new stats
####### dataframe in the next cell.

statsL=[]
for name, df in d.items():
    df_=df.iloc[:,2:].dropna(how='all')
        
    for row in df_.iterrows():
        indx, data = row
        dfs=pd.DataFrame(np.array(data)).transpose()
        
        sc = SedSASClass.SedSAS( str(name), dfs, scrns) 
        gs=sc.ComputeFWLogarithmicGraphicStats()
    
        statsL.append(gs)

    # convert the stats list for the current transect, with all the transect stats inside,
    # to a numpy array, then compute the mean down each column
    # mean: stats[0]; sorting stats[1];  skew: stats[2];  Kurtosis: stats[3]
    statsA=np.mean( np.array(statsL), axis=0 )

df_stats=pd.DataFrame(statsL)

print('Done')    

----------------------------------------------------------------------
Particle-Size Composition Analysis. Processing Sample ID: 201504
----------------------------------------------------------------------
----------------------------------------------------------------------
Particle-Size Composition Analysis. Processing Sample ID: 201504
----------------------------------------------------------------------
----------------------------------------------------------------------
Particle-Size Composition Analysis. Processing Sample ID: 201504
----------------------------------------------------------------------
----------------------------------------------------------------------
Particle-Size Composition Analysis. Processing Sample ID: 201504
----------------------------------------------------------------------
----------------------------------------------------------------------
Particle-Size Composition Analysis. Processing Sample ID: 201504
------------------------------------

### Final "Pooled" Statistics for the Mean, Sorting, Skewness, and Kurtosis - Units: $\phi$

In [34]:
df_stats=pd.DataFrame(statsA).transpose()
df_stats.columns=['Mean','Sorting','Skewness','Kurtosis']
print('Pooled global statistics for all surveys: 2014 thru 2017. Units=phi')
df_stats.to_csv('/Users/paulp/GoogleDrive/projects/PeaIslandBeachMonitoring/data/StatsGlobal.csv')
df_stats

Pooled global statistics for all surveys: 2014 thru 2017. Units=phi


,Mean,Sorting,Skewness,Kurtosis
0,0.872694,0.769301,-0.05698,1.05158


### Final "Pooled" Statistics for the Mean and Sorting - Units: mm

In [33]:
mmm=round( 2**(-1*(df_stats['Mean'][0])), 3)
sdmm=round( 2**(-1*(df_stats['Sorting'][0])), 3) 

print( 'Pooled Global Mean for all Surveys 2014 - 2017:', mmm, 'mm' )
print( 'Pooled Global Sorting for all Surveys 2014 - 2017:',sdmm, 'mm' )

Pooled Global Mean for all Surveys 2014 - 2017: 0.546 mm
Pooled Global Sorting for all Surveys 2014 - 2017: 0.587 mm


In [30]:
2**(-1*(df_stats['Mean'][0]))

0.54612619769943949

### The End

### Spare Parts:

In [102]:
### Compute statistics for each transect in survey:
scrns=[-1.0,-0.5,0.0,0.5,1.0,1.25,1.5,1.75,2.0,2.5,3.0,3.5,4.0,5.0]
transects=['C11','C10','C9','C8','C7','C6','C5','C4','C3','C2','C1','T1','T2','T3','T4','T5','T6','T7','T8','T9','T10',
'T11','T12','T13','T14','T15','T16']

for transect in ['C11']:
    dfs=df.loc[df['Transect']=='C11']
    dfs=dfs.iloc[:,2:].copy()
    
    for row in dfs.iterrows():
        L=list(row)[1]
    #L=dfs.values.tolist()
        globalL.append(L)
    
        
    #sc = SedSASClass.SedSAS( transect, dfs, scrns)


In [58]:
t1=[['C11','C10','C9','C8','C7','C6','C5','C4','C3','C2','C1','T1','T2','T3','T4','T5','T6','T7','T8','T9','T10',
'T11','T12','T13','T14','T15','T16'],['C11','C10','C9','C8','C7','C6','C5','C4','C3','C2','C1','T1','T2','T3','T4','T5','T6','T7','T8','T9','T10',
'T11','T12','T13','T14','T15','T16'],['C11','C10','C9','C8','C7','C6','C5','C4','C3','C2','C1','T1','T2','T3','T4','T5','T6','T7','T8','T9','T10',
'T11','T12','T13','T14','T15','T16']]

dftmp=pd.DataFrame(t1)
dftmp

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,C11,C10,C9,C8,C7,C6,C5,C4,C3,C2,...,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16
1,C11,C10,C9,C8,C7,C6,C5,C4,C3,C2,...,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16
2,C11,C10,C9,C8,C7,C6,C5,C4,C3,C2,...,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16


In [50]:
dfstat=pd.DataFrame(index=np.arange(0, len(df)),columns=['Position','Q1','Median','Q3','Mean','SD','Skew','Kurt'])
dfstat
len(df)
n=94
dfs=df.iloc[n:n+1,2:17].copy()
dfs

,phi_-1,"phi_-0,5",phi_0,"phi_0,5",phi_1,"phi_1,25","phi_1,5","phi_1,75",phi_2,"phi_2,5",phi_3,"phi_3,5",phi_4,remainder
94,0.2,0.14,0.18,0.7,4.85,5.69,9.43,10.64,15.98,12.11,2.52,0.55,0.0,0.01


In [51]:

for n in range(0, len(df)):  #, sample in enumerate(samples):
    L=[]
    print('Processing sample:', n)
    L.append( df['Transect'][n] )
    L.append( df['Sample'][n] )
    dfs=df.iloc[n:n+1,2:17].copy()
    sc = SedSASClass.SedSAS( df['Sample'][n], dfs, scrns)
    #print( sc.GetDataFrame() )
    #print(sc.GetQuantileList())
        
### df.iloc[0:1,3:] 
    gs=sc.ComputeFWLogarithmicGraphicStats()
    L.append(gs[0]); L.append(gs[1]);  L.append(gs[2]);  L.append(gs[3])
    #sc.PrintComputedStatistics(gs, units='phi', method='FWlog' )
    
    dflist_bu.append(L)

Processing sample: 0
----------------------------------------------------------------------
Particle-Size Composition Analysis. Processing Sample ID: S1
----------------------------------------------------------------------
Processing sample: 1
----------------------------------------------------------------------
Particle-Size Composition Analysis. Processing Sample ID: S2
----------------------------------------------------------------------
Processing sample: 2
----------------------------------------------------------------------
Particle-Size Composition Analysis. Processing Sample ID: S3
----------------------------------------------------------------------
Processing sample: 3
----------------------------------------------------------------------
Particle-Size Composition Analysis. Processing Sample ID: S4
----------------------------------------------------------------------
Processing sample: 4
----------------------------------------------------------------------
Particle-Siz

In [52]:
dflist_bu

[['T1',
  'S1',
  -0.6366666666666667,
  0.9571969696969698,
  0.19958643507030602,
  0.9552569949463824],
 ['T1',
  'S2',
  0.7933333333333333,
  0.7403030303030302,
  -0.09351482957060639,
  1.060503633598107],
 ['T1',
  'S3',
  1.3766666666666667,
  0.5657575757575757,
  -0.043103448275862113,
  0.9208662214126694],
 ['T1',
  'S4',
  1.3133333333333332,
  0.9600757575757575,
  -0.28612865000619003,
  1.7119193989071038],
 ['T10',
  'S1',
  0.68,
  0.9554545454545456,
  0.0758303321328532,
  0.8367486338797815],
 ['T10',
  'S2',
  0.84,
  1.0178030303030303,
  -0.11666055315090613,
  0.806944303024132],
 ['T10',
  'S3',
  1.6733333333333331,
  0.7296969696969697,
  -0.27948542805100174,
  1.1111111111111112],
 ['T10',
  'S4',
  0.47666666666666674,
  0.959469696969697,
  -0.10673832162050441,
  0.8224601315936212],
 ['T7',
  'S1',
  -0.6966666666666667,
  0.4290151515151515,
  0.18796992481203015,
  0.8935769954313356],
 ['T7',
  'S2',
  0.5433333333333333,
  1.0371969696969696,
  0.

In [ ]:
dflist.extend(dflist_bu)
len(dflist)

In [ ]:
### Create dataframe containing results from sedSAS:
df2=pd.DataFrame(dflist, columns=['Transect','Sample','Mean','Std','Skew','Kurt'] )

### Add a field GRP (Group) to df identifying each record as belonging to a control or  
### treatment transect (Cnn vs Tnn):
## to do this parse the values in the Slice field...
df2.loc[df2['Transect'].str[0] == 'C', 'GRP'] ='control'
df2.loc[df2['Transect'].str[0] =='T', 'GRP'] = 'treatment'

df2

In [ ]:
pvttbl_201708=pd.pivot_table(df2, index=['Sample'], values=['Mean','Std'], aggfunc=[np.mean])#.mean()
pvttbl_201708

#pvttbl_201407
#pvttbl_201409
#pvttbl_201504
#pvttbl_201508
#pvttbl_201510
#pvttbl_201602
#pvttbl_201605
#pvttbl_201608
#pvttbl_201610
#pvttbl_201702
#pvttbl_201704
pvttbl_201708

In [ ]:
pvttbl_201708['Survey']=20178
pvttbl_201708

In [ ]:
pts = [pvttbl_201407,pvttbl_201409,pvttbl_201504,pvttbl_201508,pvttbl_201510,
      pvttbl_201602,pvttbl_201605,pvttbl_201608,pvttbl_201610,pvttbl_201702,
      pvttbl_201704,pvttbl_201708]

samples=['S1','S2','S3','S4']  

#samples=['S1','S2','S3','S4','S5']  


pvttbl_201407['Survey']
pvttbl_201409['Survey']
pvttbl_201504.loc[sample][0]
pvttbl_201508.loc[sample][0]
pvttbl_201510.loc[sample][0]
pvttbl_201602.loc[sample][0]
pvttbl_201605.loc[sample][0]
pvttbl_201608.loc[sample][0]
pvttbl_201610.loc[sample][0]
pvttbl_201702.loc[sample][0]
pvttbl_201704.loc[sample][0]
pvttbl_201708.loc[sample][0]


        print(n, sample, s1, s2)
    except KeyError as e:
        print('Oops!', e)

    #.loc['S1'][1]    len(pvttbl_201407)

In [ ]:
pts = [pvttbl_201407,pvttbl_201409,pvttbl_201504,pvttbl_201508,pvttbl_201510,
      pvttbl_201602,pvttbl_201605,pvttbl_201608,pvttbl_201610,pvttbl_201702,
      pvttbl_201704,pvttbl_201708]

samples=['S1','S2','S3','S4']  
#samples=['S1','S2','S3','S4','S5']  
mean_sum=0;   std_sum=0

df_concat = pd.concat(pts, axis=0)

df_concat.mean()  #.describe() #mean()

In [ ]:
print('For S1:')
print(df_concat.loc['S1']['mean'].Mean.mean() )
print(df_concat.loc['S1']['mean'].Std.mean() )
print('')

print('For S2:')
print(df_concat.loc['S2']['mean'].Mean.mean() )
print(df_concat.loc['S2']['mean'].Std.mean() )
print('')

print('For S3:')
print(df_concat.loc['S3']['mean'].Mean.mean() )
print(df_concat.loc['S3']['mean'].Std.mean() )
print('')

print('For S4:')
print(df_concat.loc['S4']['mean'].Mean.mean() )
print(df_concat.loc['S4']['mean'].Std.mean() )
print('')

print('For S5:')
print(df_concat.loc['S5']['mean'].Mean.mean() )
print(df_concat.loc['S5']['mean'].Std.mean() )
print('')

(df_concat.loc['S1']['mean'].Mean.mean()+df_concat.loc['S2']['mean'].Mean.mean()+
df_concat.loc['S3']['mean'].Mean.mean()+df_concat.loc['S4']['mean'].Mean.mean()+
df_concat.loc['S5']['mean'].Mean.mean())/5

In [ ]:
df_concat

In [ ]:
dfp=df.iloc[:,4:17]
print('Global Mean Grain Size:',dfp.mean(axis=0).mean() )
print('Global Grain Size Standard Deviation (Sorting):',dfp.std(axis=0).std() )

In [ ]:
fp='/Users/paulp/GoogleDrive/projects/PeaIslandBeachMonitoring/data/FinalDataSets/'
fns=['PINWR_201407_GrainSize.xlsx','PINWR_201409_GrainSize.xlsx','PINWR_201504_GrainSize.xlsx',
   'PINWR_201508_GrainSize.xlsx','PINWR_201510_GrainSize.xlsx','PINWR_201602_GrainSize.xlsx',
   'PINWR_201605_GrainSize.xlsx','PINWR_201608_GrainSize.xlsx','PINWR_201610_GrainSize.xlsx',
   'PINWR_201702_GrainSize.xlsx','PINWR_201704_GrainSize.xlsx','PINWR_201708_GrainSize.xlsx']
delim=','
samples=['S1','S2','S3','S4','S5']  
scrns=[-1.0,-0.5,0.0,0.5,1.0,1.25,1.5,1.75,2.0,2.5,3.0,3.5,4.0]

###############################################################################

# load the first sheet in the Excel ss file (header=1 allows for existing
df = pd.read_excel(fp+'PINWR_201407_GrainSize.xlsx', header=0)


# column header row in spreadsheet:
for fn in fns:
    dft=pd.read_excel(fp+fn, header=0)
    df=pd.concat([df,dft])
    
# since we only need a subset of the total file content, and that content is
# not contiguous, we extract what we want/need based on the column names
# in the source Excel spreadsheet. The string items in the list are names of
# the wanted/required columns in the spreadsheet:
df=df.drop(['sheet_code','sample_date','pan_weight','pan+wet_weight','pan+dry_weight',
            'summed_weight','dry_weight'], axis=1).copy()
df=df.rename(columns = {'transect_id':'Transect', 'sample_number':'Sample'})
df=df.replace('-',0).copy()
df

In [ ]:
df.iloc[0:1,0:14]  #.mean()

<h4>Data Preprocessing II: (Aggregating individual screens into gravel and fine fractions)</h4>

In [ ]:
# compute the sample percentage that is in the gravel and fines fractions:
# 1., sum all the individual bin (phi) weights:
df['total_wt']=df['phi_-1']+df['phi_-0.5']+df['phi_0']+df['phi_0.5']+df['phi_1']+df['phi_1.25'] \
+df['phi_1.5']+df['phi_1.75']+df['phi_2']+df['phi_2.5']+df['phi_3']+df['phi_3.5']+df['remainder']

# 2., using the phi_-1 field as the 'gravel' weight fraction:
df['pgravel']=df['phi_-1']    #/df['total_wt'])*100.0

               
# 3., combine phi 2.5, 3.0, 3.5, 4.0, and pan (remainder) to create a fines field:
df['pfines']=df['phi_2.5']+df['phi_3']+df['phi_3.5']+df['remainder']   #/df['total_wt'])*100.0

# replace the NaN values in the new fields with 0s:
df=df.fillna(0)

# drop the now unnecessary fields: phi_-1, phi_-0.5, phi_0, phi_0.5, phi_1, phi_1.25, phi_1.5, phi_1.75, phi_2
# phi_2.5, phi_3, phi_3.5, phi_4, and remainder:
df=df.drop(['phi_-1','phi_-0.5','phi_0','phi_0.5','phi_1','phi_1.25','phi_1.5','phi_1.75','phi_2','phi_2.5'], axis=1)
df=df.drop(['phi_3','phi_3.5','remainder'], axis=1)



<h4>Data Preprocessing III: (Extracting records from the df for only the currently defined sample (user input: _sample_) NOTE: if you're creating the combined (.all) plot where all 4/5 transect samples are considered together) there is no need to run the search query, below)</h4>

In [ ]:
query_string = 'sample == '+'\"'+sample+'\"'
df1=df.query(query_string)
#df1=df.groupby(['survey', 'transect'], sort=False, as_index=False)[['pgravel','pfines']].sum()

#df1

<h4>Plot the transect SAMPLE Gravel Fraction (for the combined .all plot see below):</h4>

In [ ]:
####### LET'S PLOT STUFF -- THE GRAVELS #######

sns.set_context('talk')
figpg = plt.figure(figsize=(15,5))
axspg = figpg.add_subplot(1,1,1)
figpg.tight_layout(pad=5.1, w_pad=0.5, h_pad=2.0)

sns.set(style="ticks")
#sns.set(rc={'axes.facecolor':'white', 'grid.color': 'white'})
# nested boxplot showing percent gravel in samples by survey, by group
axc=sns.boxplot(x='survey', y='pgravel', hue='group', data=df1, palette='PRGn' )

axc.set(ylim=(0,100))
sns.despine(offset=10, trim=True)

plt.xticks(size=16)
plt.yticks(size=16)
axc.set_xlabel('') 
plt.ylabel('percent gravel', fontsize=20)
axc.legend_.remove()

plt.savefig(plotpath+sample+'_'+date+'_percent_gravel.jpg', dpi=300) 

<h4>Plot the SAMPLE Fines Fraction (for the combined .all plot, see below):</h4>

In [ ]:
####### LET'S PLOT STUFF -- THE FINES #######

sns.set_context('talk')
figpf = plt.figure(figsize=(15,5))
axspf = figpf.add_subplot(1,1,1)
figpf.tight_layout(pad=3.1, w_pad=0.5, h_pad=2.0)

sns.set(style="ticks")

# nested boxplot showing percent fines in samples by survey, by group
axf=sns.boxplot(x='survey', y='pfines', hue='group', data=df1, palette='PRGn' )

axf.set(ylim=(0,80))
sns.despine(offset=10, trim=True)  # despine must come AFTER the axis limits are set!!!

plt.xticks(size=16)
plt.yticks(size=16)
axf.set_xlabel('')    #plt.xlabel('survey', fontsize=14)
plt.ylabel('percent fines', fontsize=20)
axf.legend_.remove()

plt.savefig(plotpath+sample+'_'+date+'_percent_fines.jpg', dpi=300)       

<h4>Plot the combined gravel fraction (this is the gravel fraction .all plot):</h4>

In [ ]:
figpg = plt.figure(figsize=(15,10))
axspg = figpg.add_subplot(1,1,1)
figpg.tight_layout(pad=5.1, w_pad=0.5, h_pad=2.0)

sns.set_context('talk')
sns.set(style="ticks")
#sns.set(rc={'axes.facecolor':'white', 'grid.color': 'white'})
# nested boxplot showing percent gravel in samples by survey, by group
axc=sns.boxplot(x='survey', y='pgravel', hue='group', data=df, palette='PRGn' )

axc.set(ylim=(0,100))
sns.despine(offset=10, trim=True)

plt.xticks(size=18)
plt.yticks(size=18)
#axc.set_xlabel('') 
plt.xlabel('survey', fontsize=28)
plt.ylabel('percent gravel', fontsize=28)
plt.legend(title='Sampling Group',fontsize=18, loc='best')

plt.savefig(plotpath+'percent_gravel_all.jpg', dpi=300) 

<h4>Plot the combined fines fraction (this is the fine-fraction .all plot):</h4>

In [ ]:
sns.set_context('talk')
figpf = plt.figure(figsize=(15,10))
axspf = figpf.add_subplot(1,1,1)
figpf.tight_layout(pad=5.1, w_pad=0.5, h_pad=2.0)

sns.set(style="ticks")

# nested boxplot showing percent fines in samples by survey, by group
axf=sns.boxplot(x='survey', y='pfines', hue='group', data=df, palette='PRGn' )

axf.set(ylim=(0,80))
sns.despine(offset=10, trim=True)  # despine must come AFTER the axis limits are set!!!

plt.xticks(size=18, rotation=0)
plt.yticks(size=18)
#axf.set_xlabel('')    #
plt.xlabel('survey', fontsize=28)
plt.ylabel('percent fines', fontsize=28)
#axf.legend_.remove()
plt.legend(title='Sampling Group', fontsize=18, loc='best')

plt.savefig(plotpath+'percent_fines_all.jpg', dpi=300)       

In [21]:
A=np.random.randint(1,10,100)
B=np.random.randint(1,10, 100)
print(A.mean(), B.mean())
print('')
print( (A.mean()+B.mean()) /2 )

C=np.concatenate((A, B), axis=0)

C.mean()

4.63 4.92

4.775


4.7750000000000004